<a href="https://colab.research.google.com/github/ShreyasheeSinha/Determining-Robustness-of-NLU-Models/blob/main/analysis_notebooks/Prediciton_change_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook with code to carry out analysis for questions R1, and R2.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/MyDrive/paraphraser_work/model_predictions.tar.gz model_predictions.tar.gz

In [ ]:
! gzip -d model_predictions.tar.gz

In [ ]:
! tar -xvf  'model_predictions.tar'

model_predictions/
model_predictions/MNLI_train_RTE_dev/
model_predictions/MNLI_train_RTE_dev/roberta-base/
model_predictions/MNLI_train_RTE_dev/roberta-base/roberta_base_2_class_rte_test_p_predictions.csv
model_predictions/MNLI_train_RTE_dev/roberta-base/roberta_base_2_class_hypo_only_rte_test_p_predictions.csv
model_predictions/MNLI_train_RTE_dev/roberta-base/roberta_base_3_class_rte_test_p_predictions.csv
model_predictions/MNLI_train_RTE_dev/roberta-base/roberta_base_3_class_hypo_only_rte_test_p_predictions.csv
model_predictions/MNLI_train_RTE_dev/roberta/
model_predictions/MNLI_train_RTE_dev/roberta/roberta-base/
model_predictions/MNLI_train_RTE_dev/roberta/roberta-base/roberta_base_3_class_rte_test_predictions.csv
model_predictions/MNLI_train_RTE_dev/roberta/roberta-base/roberta_base_3_class_rte_dev_predictions.csv
model_predictions/MNLI_train_RTE_dev/roberta/roberta-base/roberta_base_3_class_hypo_only_rte_dev_predictions.csv
model_predictions/MNLI_train_RTE_dev/roberta/roberta-ba

In [ ]:
%cd /content/model_predictions/MNLI_train_RTE_paraphrased_dev/cbow

/content/model_predictions/MNLI_train_RTE_paraphrased_dev/cbow


In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
def read_jsonl_file(path):
  with open(path, 'r') as json_file:
      json_list = list(json_file)

  results = []
  for json_str in json_list:
      results.append(json.loads(json_str))
  return results

In [ ]:
test_data = pd.DataFrame(read_jsonl_file("/content/gdrive/MyDrive/paraphraser_work/Final_datasets/rte_test_p.jsonl")) # Unannotated dataset: /content/gdrive/MyDrive/paraphraser_work/MTurk data/Generated_dataset/rte_test_paraphrased_p_d_or_h_d_sorted.jsonl
test_data = test_data.fillna(value=np.nan)

In [ ]:
ph_ = test_data.copy()
p_h = test_data.copy()
p_h_ = test_data.copy()

p_h = p_h.loc[~p_h['s1_para_id'].isnull() & p_h['s2_para_id'].isnull()]
ph_ = ph_.loc[ph_['s1_para_id'].isnull() & ~ph_['s2_para_id'].isnull()]
p_h_ = p_h_.loc[~p_h_['s1_para_id'].isnull() & ~p_h_['s2_para_id'].isnull()]

In [ ]:
print(len(p_h), len(ph_), len(p_h_))

415 631 132


In [ ]:
predictions = pd.read_csv('cbow_3_class_rte_test_p_predictions.csv')
predictions

,Unnamed: 0,dataset,corpus_sent_id,gold_label,task,sentence1,sentence2,s2_para_id,silver_label,s1_para_id,prediction
0,0,RTE1_test.column,1066,1,RC,"The biggest fires on Mount Graham , the lightn...",The Nuttall and Gibson fires were caused by li...,NaN,NaN,NaN,1
1,1,RTE3_test.column,767,0,SUM,There has been some speculation in the Russian...,Mr Putin is a vehement critic of Mr Litvinenko .,NaN,NaN,NaN,1
2,2,RTE1_test.column,39,1,PP,Mr. Clinton received a hefty advance for the b...,Bill Clinton received a reported $ 10 million ...,NaN,NaN,NaN,1
3,3,RTE2_test.column,485,1,SUM,Insurgent attacks on pipelines have brought oi...,Insurgents have launched attacks on Iraq 's oi...,NaN,NaN,NaN,1
4,4,RTE2_test.column,485,1,SUM,Insurgent attacks on pipelines have brought oi...,Insurgents have launched attacks on Iraq's oil...,RTE2_test.column_s2_485_para1,True,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
3986,3986,RTE3_test.column,214,0,IR,Almost a century of French history has come to...,French conscientious objectors are given speci...,RTE3_test.column_s2_214_para1,False,NaN,1
3987,3987,RTE3_test.column,214,0,IR,Almost a century of French history has come to...,French conscientious objectors are given speci...,RTE3_test.column_s2_214_para2,False,NaN,1
3988,3988,RTE3_test.column,214,0,IR,Almost a century of French history has come to...,French Observers are given special tasks.,RTE3_test.column_s2_214_para3,False,NaN,1
3989,3989,RTE3_test.column,526,1,QA,The most recent underwater volcano eruption wa...,Earthquakes have the potential to generate a t...,NaN,NaN,NaN,1


In [ ]:
# THIS IS KEY TO OUR WORK, DO NOT ALTER
df = pd.DataFrame()

for index, row in test_data.iterrows():
  dataset = row['dataset']
  corpus_sent_id = row['corpus_sent_id']
  s1_para_id = row['s1_para_id']
  s2_para_id = row['s2_para_id']
  prediction_rows = predictions[(predictions['dataset'] == dataset) & (predictions['corpus_sent_id'] == corpus_sent_id)]
  if pd.isnull(s1_para_id) and pd.isnull(s2_para_id):
    prediction_rows = prediction_rows[(prediction_rows['s1_para_id'].isnull()) & (prediction_rows['s2_para_id'].isnull())]
  elif pd.isnull(s1_para_id):
    prediction_rows = prediction_rows[(prediction_rows['s1_para_id'].isnull()) & (prediction_rows['s2_para_id'] == s2_para_id)]
  elif pd.isnull(s2_para_id):
    prediction_rows = prediction_rows[(prediction_rows['s1_para_id'] == s1_para_id) & (prediction_rows['s2_para_id'].isnull())]
  else:
    prediction_rows = prediction_rows[(prediction_rows['s1_para_id'] == s1_para_id) & (prediction_rows['s2_para_id'] == s2_para_id)]
  df = df.append(prediction_rows)

In [ ]:
df

,Unnamed: 0,dataset,corpus_sent_id,gold_label,task,sentence1,sentence2,s2_para_id,silver_label,s1_para_id,prediction
551,551,RTE1_test.column,25,0,IR,"The film was the evening 's big winner , baggi...",Two films won six Oscars .,NaN,NaN,NaN,1
552,552,RTE1_test.column,25,0,IR,"The film was the big winner of the evening, wi...",Two films won six Oscars .,NaN,False,RTE1_test.column_s1_25_para1,0
1005,1005,RTE1_test.column,49,1,IR,Kidnappings in Argentina have increased more t...,Argentina sees upsurge in kidnappings .,NaN,NaN,NaN,1
1006,1006,RTE1_test.column,49,1,IR,Kidnappings in Argentina have increased more t...,Argentine sees upsurge in kidnappings.,RTE1_test.column_s2_49_para1,True,NaN,1
2417,2417,RTE1_test.column,75,1,IR,"At the close , the Dow Jones industrial averag...",Dow Jones is down .,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2236,2236,RTE3_test.column,796,1,SUM,It has pledged to allow the yuan to float more...,China has an export-driven economy.,RTE3_test.column_s2_796_para1,True,NaN,1
2934,2934,RTE3_test.column,798,0,SUM,"The world 's largest consumer of oil , US impo...",There was a conflict between Israel and Lebanon .,NaN,NaN,NaN,1
2935,2935,RTE3_test.column,798,0,SUM,"The world's largest consumer of oil, US import...",There was a conflict between Israel and Lebanon .,NaN,False,RTE3_test.column_s1_798_para1,1
3043,3043,RTE3_test.column,800,0,SUM,It 's very difficult to get teams from China t...,It is difficult to get the right to stay in Ch...,NaN,NaN,NaN,1


In [ ]:
df.drop("Unnamed: 0", axis=1, inplace=True)
df

,dataset,corpus_sent_id,gold_label,task,sentence1,sentence2,s2_para_id,silver_label,s1_para_id,prediction
551,RTE1_test.column,25,0,IR,"The film was the evening 's big winner , baggi...",Two films won six Oscars .,NaN,NaN,NaN,1
552,RTE1_test.column,25,0,IR,"The film was the big winner of the evening, wi...",Two films won six Oscars .,NaN,False,RTE1_test.column_s1_25_para1,0
1005,RTE1_test.column,49,1,IR,Kidnappings in Argentina have increased more t...,Argentina sees upsurge in kidnappings .,NaN,NaN,NaN,1
1006,RTE1_test.column,49,1,IR,Kidnappings in Argentina have increased more t...,Argentine sees upsurge in kidnappings.,RTE1_test.column_s2_49_para1,True,NaN,1
2417,RTE1_test.column,75,1,IR,"At the close , the Dow Jones industrial averag...",Dow Jones is down .,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
2236,RTE3_test.column,796,1,SUM,It has pledged to allow the yuan to float more...,China has an export-driven economy.,RTE3_test.column_s2_796_para1,True,NaN,1
2934,RTE3_test.column,798,0,SUM,"The world 's largest consumer of oil , US impo...",There was a conflict between Israel and Lebanon .,NaN,NaN,NaN,1
2935,RTE3_test.column,798,0,SUM,"The world's largest consumer of oil, US import...",There was a conflict between Israel and Lebanon .,NaN,False,RTE3_test.column_s1_798_para1,1
3043,RTE3_test.column,800,0,SUM,It 's very difficult to get teams from China t...,It is difficult to get the right to stay in Ch...,NaN,NaN,NaN,1


In [ ]:
def task_r1(df):
  grouped_df = df.groupby(["dataset", "corpus_sent_id"])
  number_of_preds_different_from_ph = 0
  total = 0
  for name, group in grouped_df:
    ph_pred = group[group['s1_para_id'].isnull() & group['s2_para_id'].isnull()]["prediction"].to_list()[0]
    paraphrased_preds = group[~(group['s1_para_id'].isnull() & group['s2_para_id'].isnull())]["prediction"].to_list()
    total += len(paraphrased_preds)
    for paraphrased_pred in paraphrased_preds:
      if paraphrased_pred != ph_pred:
        number_of_preds_different_from_ph += 1
  print(total)
  return number_of_preds_different_from_ph, number_of_preds_different_from_ph/total * 100

In [ ]:
number_of_different_preds, percentage_different_preds = task_r1(df)
print(number_of_different_preds, percentage_different_preds)

1126
172 15.2753108348135


In [ ]:
def task_r11(df):
  grouped_df = df.groupby(["dataset", "corpus_sent_id"])
  number_of_preds_different_from_ph = 0
  total = 0
  for name, group in grouped_df:
    ph_pred = group[group['s1_para_id'].isnull() & group['s2_para_id'].isnull()]["prediction"].to_list()[0]
    gold_label = group[group['s1_para_id'].isnull() & group['s2_para_id'].isnull()]["gold_label"].to_list()[0]
    if ph_pred == gold_label:
      paraphrased_preds = group[~(group['s1_para_id'].isnull() & group['s2_para_id'].isnull())]["prediction"].to_list()
      total += len(paraphrased_preds)
      for paraphrased_pred in paraphrased_preds:
        if paraphrased_pred != ph_pred:
          number_of_preds_different_from_ph += 1
  print(total)
  return number_of_preds_different_from_ph, number_of_preds_different_from_ph/total * 100

In [ ]:
number_of_different_preds, percentage_different_preds = task_r11(df)
print(number_of_different_preds, percentage_different_preds)

608
82 13.486842105263158


In [ ]:
def task_r12(df):
  grouped_df = df.groupby(["dataset", "corpus_sent_id"])
  number_of_preds_different_from_ph = 0
  total = 0
  for name, group in grouped_df:
    ph_pred = group[group['s1_para_id'].isnull() & group['s2_para_id'].isnull()]["prediction"].to_list()[0]
    gold_label = group[group['s1_para_id'].isnull() & group['s2_para_id'].isnull()]["gold_label"].to_list()[0]
    if ph_pred != gold_label:
      paraphrased_preds = group[~(group['s1_para_id'].isnull() & group['s2_para_id'].isnull())]["prediction"].to_list()
      total += len(paraphrased_preds)
      for paraphrased_pred in paraphrased_preds:
        if paraphrased_pred != ph_pred:
          number_of_preds_different_from_ph += 1
  print(total)
  return number_of_preds_different_from_ph, number_of_preds_different_from_ph/total * 100

In [ ]:
number_of_different_preds, percentage_different_preds = task_r12(df)
print(number_of_different_preds, percentage_different_preds)

518
90 17.374517374517374


In [ ]:
def task_r2(df):
  grouped_df = df.groupby(["dataset", "corpus_sent_id"])
  number_of_both_paraphrased_preds_different_from_ph = 0
  number_of_one_paraphrased_preds_different_from_ph = 0
  total_both_paraphrased_preds = 0
  total_one_paraphrased_preds = 0
  for name, group in grouped_df:
    ph_pred = group[group['s1_para_id'].isnull() & group['s2_para_id'].isnull()]["prediction"].to_list()[0]
    paraphrased_preds = group[~(group['s1_para_id'].isnull() & group['s2_para_id'].isnull())]
    both_paraphrased_preds = paraphrased_preds[~paraphrased_preds['s1_para_id'].isnull() & ~paraphrased_preds['s2_para_id'].isnull()]["prediction"].to_list()
    one_paraphrased_preds = paraphrased_preds[~(~paraphrased_preds['s1_para_id'].isnull() & ~paraphrased_preds['s2_para_id'].isnull())]["prediction"].to_list()

    total_both_paraphrased_preds += len(both_paraphrased_preds)
    for paraphrased_pred in both_paraphrased_preds:
      if paraphrased_pred != ph_pred:
        number_of_both_paraphrased_preds_different_from_ph += 1
    
    total_one_paraphrased_preds += len(one_paraphrased_preds)
    for paraphrased_pred in one_paraphrased_preds:
      if paraphrased_pred != ph_pred:
        number_of_one_paraphrased_preds_different_from_ph += 1

  print(total_both_paraphrased_preds, total_one_paraphrased_preds)
  return number_of_both_paraphrased_preds_different_from_ph, number_of_both_paraphrased_preds_different_from_ph/total_both_paraphrased_preds * 100, number_of_one_paraphrased_preds_different_from_ph, number_of_one_paraphrased_preds_different_from_ph/total_one_paraphrased_preds * 100

In [ ]:
task_r2(df)

122 1004


(23, 18.852459016393443, 149, 14.840637450199203)